In [1]:
import pandas as pd
import random

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
import joblib

In [2]:
random.seed(42)

In [3]:
df = pd.read_csv('skills.csv')

In [4]:
X = df['skills']
y = df['profession']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [6]:
vectorizer = TfidfVectorizer(token_pattern=r'[^,]+', lowercase=True)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [7]:
model = RandomForestClassifier(
    n_estimators=100,    
    max_depth=25,        
    random_state=42,
    n_jobs=-1           
)

In [8]:
model.fit(X_train_tfidf, y_train)

RandomForestClassifier(max_depth=25, n_jobs=-1, random_state=42)

In [9]:
y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division=0)

In [10]:
accuracy

0.947

In [11]:
joblib.dump(model, 'model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

In [12]:
user_input = "HTML, CSS"

# 3. Обрабатываем текст и делаем предсказание
X_input = vectorizer.transform([user_input])
predicted_profession = model.predict(X_input)[0]

print(f"Подходящая профессия: {predicted_profession}")

Подходящая профессия: Фронтенд-разработчик


In [13]:
def smart_predict(skills_text, top1_threshold=0.3, min_prob_threshold=0.027):
    # Векторизация
    X_vec = vectorizer.transform([skills_text])
    
    # Предсказание вероятностей
    proba = model.predict_proba(X_vec)[0]
    classes = model.classes_
    
    # Сортируем вероятности по убыванию
    top_indices = proba.argsort()[::-1]
    
    top1_prob = proba[top_indices[0]]
    
    if top1_prob >= top1_threshold:
        # Если модель очень уверена — отдаем только одну профессию
        result = [(classes[top_indices[0]], round(top1_prob * 100, 2))]
    else:
        # Иначе отдаем все профессии с вероятностью выше минимального порога
        result = []
        for idx in top_indices:
            if proba[idx] >= min_prob_threshold:
                result.append((classes[idx], round(proba[idx] * 100, 2)))
                
    if not result:
        result.append((classes[top_indices[0]], round(proba[top_indices[0]] * 100, 2)))
        result.append((classes[top_indices[1]], round(proba[top_indices[0]] * 100, 2)))
        result.append((classes[top_indices[2]], round(proba[top_indices[0]] * 100, 2)))
    
    return result

# ----- Пример использования -----

# Пользовательский ввод навыков
user_skills = "Python, Машинное обучение, Анализ данных, Статистика, Работа в команде, Креативность"

# Предсказание
predictions = smart_predict(user_skills)

print("\n🎯 Предсказание на основе введенных навыков:")
for profession, probability in predictions:
    print(f"- {profession} ({probability}%)")


🎯 Предсказание на основе введенных навыков:
- Инженер машинного обучения (7.38%)
- Data Scientist (3.67%)
- Продуктовый менеджер (IT) (3.14%)


In [14]:
user_skills = "Python, Машинное обучение, Pandas, Статистика,"

# Предсказание
predictions = smart_predict(user_skills)

print("\n🎯 Предсказание на основе введенных навыков:")
for profession, probability in predictions:
    print(f"- {profession} ({probability}%)")


🎯 Предсказание на основе введенных навыков:
- Инженер машинного обучения (6.35%)
- Data Scientist (6.08%)
- Аналитик данных (3.57%)


In [15]:
user_skills = "Kotlin, Android Studio, Jetpack Compose"

# Предсказание
predictions = smart_predict(user_skills)

print("\n🎯 Предсказание на основе введенных навыков:")
for profession, probability in predictions:
    print(f"- {profession} ({probability}%)")


🎯 Предсказание на основе введенных навыков:
- Мобильный разработчик (3.71%)


In [16]:
model

RandomForestClassifier(max_depth=25, n_jobs=-1, random_state=42)